In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source,'lxml')

In [9]:
column_names = ['PostalCode','Borough','Neighborhood']

neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,PostalCode,Borough,Neighborhood


In [10]:
neigh_loc = soup.find('table', class_='wikitable sortable').find_all('tr')[1:]
i = 0
for row in neigh_loc:
    cells = row.find_all("td")
    if cells[1].text != "Not assigned":
        if cells[2].text != "Not assigned\n":
            neighborhoods.loc[i] = [cells[0].text, cells[1].text, cells[2].text[0:len(cells[2].text)-1]]
        else:
            neighborhoods.loc[i] = [cells[0].text, cells[1].text, cells[1].text]
        i = i + 1
        
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [12]:
combined_neighbors = neighborhoods.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

combined_neighbors.shape


(103, 3)

In [18]:
locations = pd.read_csv("https://cocl.us/Geospatial_data")

locations.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
neighbor_locations = pd.concat([combined_neighbors, locations], axis = 1, join='inner')
neighbor_locations = neighbor_locations.T.drop_duplicates().T
neighbor_locations.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648


In [32]:
neighbor_locations.shape

(103, 5)